In [1]:
%cd "~/Skorch/src/MoNet/"

/home/myf/Skorch/src/MoNet


普通方法

- 书写计算过程可读性差，括号太多
  该代码定义了一个具有三个全连接层和 PReLU 激活函数的神经网络。然后，它将该网络应用于输入张量并计算三个不同的输出张量。

步骤如下：

1. 导入 torch 库。
2. 使用三个全连接层定义神经网络架构：Fc1、Fc2 和 Fc3。
3. 使用 \_parameters.values() 方法提取每个层的权重和偏置参数，并将它们分配给变量 W1、b1、W2、b2、W3 和 b3。
4. 创建 PReLU 激活函数的实例并将其分配给变量 f。
5. 创建一个具有值 [1、2、3] 的输入张量 x 并将其转换为 float 数据类型。
6. 使用矩阵乘法和激活函数将网络应用于输入张量，计算输出张量 y1。
7. 直接使用全连接层和激活函数将网络应用于输入张量，计算输出张量 y2。
8. 使用 Sequential 容器将层和激活函数组合起来，将网络应用于输入张量，计算输出张量 y3。
9. 打印 y1、y2 和 y3 的值。


In [4]:
import torch
# 定义一个神经网络 Net = 2(FcAct)Fc
W1, b1 = (Fc1 := torch.nn.Linear(3, 5))._parameters.values()
W2, b2 = (Fc2 := torch.nn.Linear(5, 5))._parameters.values()
W3, b3 = (Fc3 := torch.nn.Linear(5, 1))._parameters.values()
f = torch.nn.PReLU()

x = torch.tensor([1, 2, 3]).float()
y1 = f(f(x.matmul(W1.T) + b1).matmul(W2.T) + b2).matmul(W3.T) + b3
y2 = Fc3(f(Fc2(f(Fc1(x)))))
y3 = torch.nn.Sequential(Fc1, f, Fc2, f, Fc3)(x)
y1, y2, y3

(tensor([0.1832], grad_fn=<AddBackward0>),
 tensor([0.1832], grad_fn=<ViewBackward0>),
 tensor([0.1832], grad_fn=<ViewBackward0>))

MoNet 方法

- 书写计算过程可读性好，括号少, 直接用乘法表示顺序计算关系, 同时兼容普通方式调用方法
- 提供 Layer,SeqLayer,Cell,SeqCell,Mix 接口快速构建网络

代码的步骤解释如下：

1. 导入 monet 库。
2. 使用 Layer 函数定义了一个具有 3 个输入和 5 个输出的全连接层 Fc1。
3. 使用 Layer 函数定义了一个具有 5 个输入和 5 个输出的全连接层 Fc2。
4. 使用 Layer 函数定义了一个具有 5 个输入和 1 个输出的全连接层 Fc3。
5. 使用 Layer 函数定义了一个激活函数层 f。
6. 使用 torch.tensor 函数创建了一个包含元素[1,2,3]的张量 x，并将其转换为浮点型。
7. 将输入张量 x 通过层序列（Fc1\*f\*Fc2\*f\*Fc3）进行前向传播计算，得到输出张量 y。
8. 输出张量 y。


In [12]:
from macronet import *

Fc1 = Layer(3, 5)
Fc2 = Layer(5, 5)
Fc3 = Layer(5, 1)
f = Layer(net='act')

x = torch.tensor([1, 2, 3]).float()
y1 = (Fc1 * f * Fc2 * f * Fc3)(x)  # 看起来最直观
y2 = torch.nn.Sequential(Fc1, f, Fc2, f, Fc3)(x)
y3 = Mix(3, [[5, 5], 1], [['fc', 'act'], 'fc'])(x)  #一行代码即可构建完整神经网络

y1, y2, y3

(tensor([-0.0311], grad_fn=<ViewBackward0>),
 tensor([-0.0311], grad_fn=<ViewBackward0>),
 tensor([-0.5666], grad_fn=<ViewBackward0>))

自适应调整输入维度


In [3]:
# 把需要自适应的输入维度设为0即可，代入数据后会自行调整
from macronet import *

F = SeqLayer(0) * Layer(0, 1)
print(F)
F(torch.randn(2, 5)), F

SEQ>MoNet(
  (Net): Sequential(
    (0): SEQ>SeqLayer(
      (Net): Sequential(
        (fc_1_0): Linear(in_features=0, out_features=64, bias=True)
        (fc_1_1): Linear(in_features=64, out_features=64, bias=True)
      )
    )
    (1): SEQ>Layer(
      (Net): Linear(in_features=0, out_features=1, bias=True)
    )
  )
)


(tensor([[-0.0964],
         [ 0.1891]], grad_fn=<AddmmBackward0>),
 SEQ>MoNet(
   (Net): Sequential(
     (0): SEQ>SeqLayer(
       (Net): Sequential(
         (fc_1_0): Linear(in_features=5, out_features=64, bias=True)
         (fc_1_1): Linear(in_features=64, out_features=64, bias=True)
       )
     )
     (1): SEQ>Layer(
       (Net): Linear(in_features=64, out_features=1, bias=True)
     )
   )
 ))

In [3]:
from macronet import *

F = Mix(0) * SeqCell(0) * Cell(0) * SeqLayer(0) * Layer(0, 1)
F(torch.randn(2, 5))

tensor([[-0.0631],
        [-0.0631]], grad_fn=<AddmmBackward0>)

与nn.Module一起使用, 减少代码量的同时灵活处理信息流动

In [10]:
from macronet import *

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.cnn_img = Mix(0, [[16, 32], 1],
                           [['cv2', 'bn2', 'mp2', 'act'], 'fl']).Net
        self.txt_fc = Mix(0, [[32, 64]], [['fc', 'bn', 'act', 'dp']]).Net
        self.out_mlp = Mix(0, [1], ['fc']).Net

    def forward(self, img, x):
        y1 = self.cnn_img(img)
        y2 = self.txt_fc(x)
        y3 = torch.cat([y1, y2], dim=1)
        y = self.out_mlp(y3)
        return y


F = Net()
print(F)
F(torch.randn(2, 1, 64, 64), torch.randn(2, 10)), F

Net(
  (cnn_img): Sequential(
    (0:input): SeqCell(
      (Net): Sequential(
        (0): Cell(
          (Net): Sequential(
            (0:cv): Conv2d(0, 16, kernel_size=(3, 3), stride=(1, 1))
            (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (3:act): PReLU(num_parameters=1)
          )
        )
        (1): Cell(
          (Net): Sequential(
            (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
            (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (3:act): PReLU(num_parameters=1)
          )
        )
      )
    )
    (1:output): Layer(
      (Net): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (txt_fc): Sequential(
    (0:input): SeqCell(
      (Net): Sequenti

(tensor([[ 0.7318],
         [-0.5621]], grad_fn=<AddmmBackward0>),
 Net(
   (cnn_img): Sequential(
     (0:input): SeqCell(
       (Net): Sequential(
         (0): Cell(
           (Net): Sequential(
             (0:cv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
             (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
             (3:act): PReLU(num_parameters=1)
           )
         )
         (1): Cell(
           (Net): Sequential(
             (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
             (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
             (3:act): PReLU(num_parameters=1)
           )
         )
       )
     )
     (1:output): Layer(
       (Net): Flatten(start_dim=1,

输入相同时，可直接用加法将多层神经网络输出进行合并

In [15]:
from macronet import *
F = (Mix(0, [[16, 32], 1],[['cv2', 'bn2', 'mp2', 'act'], 'fl'])+
    Mix(0, [[32, 64], 1], [['fc', 'act', 'dp'],'fl']))*Layer(0)
print(F)
F(torch.randn(2, 1, 64, 64)), F

MoNet(
  (Net): Sequential(
    (0): MoNet(
      (Net): ModuleList(
        (0): Mix(
          (Net): Sequential(
            (0:input): SeqCell(
              (Net): Sequential(
                (0): Cell(
                  (Net): Sequential(
                    (0:cv): Conv2d(0, 16, kernel_size=(3, 3), stride=(1, 1))
                    (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                    (3:act): PReLU(num_parameters=1)
                  )
                )
                (1): Cell(
                  (Net): Sequential(
                    (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
                    (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                    (3:

(tensor([[-0.0351],
         [ 0.4670]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): MoNet(
       (Net): ModuleList(
         (0): Mix(
           (Net): Sequential(
             (0:input): SeqCell(
               (Net): Sequential(
                 (0): Cell(
                   (Net): Sequential(
                     (0:cv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
                     (1:bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                     (2:mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                     (3:act): PReLU(num_parameters=1)
                   )
                 )
                 (1): Cell(
                   (Net): Sequential(
                     (0:cv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
                     (1:bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                     (2:mp): MaxPool2d

乘法计算满足结合律与交换律

In [21]:
from macronet import *

Fc = Cell(1, 5, ['fc', 'act'])
Hid1 = Cell(5, 1, ['fc', 'act'])
Hid2 = Cell(5, 1, ['fc', 'act'])
Out = Layer(2, 1, 'fc')

x = torch.tensor([[0.0]])
F1 = (Fc * Hid1 + Fc * Hid2) * Out
F2 = Fc * (Hid1 + Hid2) * Out
F1(x) == F2(x)

tensor([[True]])

首字母大写返回一个组件，首字母小写构建方法会返回所有子成员

In [25]:
from macronet import *

Cell() * cell()

MoNet(
  (Net): Sequential(
    (0): Cell(
      (Net): Sequential(
        (0:fc): Linear(in_features=10, out_features=1, bias=True)
        (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2:act): PReLU(num_parameters=1)
        (3:dp): Dropout(p=0.5, inplace=False)
      )
    )
    (1): Linear(in_features=10, out_features=1, bias=True)
    (2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): PReLU(num_parameters=1)
    (4): Dropout(p=0.5, inplace=False)
  )
)

\* 与 + 使用浅拷贝，\*\*与&使用深拷贝，\*与\*\*后面可接数字，表示拷贝次数

In [33]:
A = layer()
A2 = layer()
id(A), id(A2)

(23275250179840, 23275265044208)

In [34]:
id((A * 1)), id((A * 2).Net[0]), id((A * 2).Net[1])  # 浅拷贝

(23275250179840, 23275250179840, 23275250179840)

In [35]:
B = A * A * A2  # 浅拷贝
id(B.Net[0]), id(B.Net[1]), id(B.Net[2])

(23275250179840, 23275250179840, 23275265044208)

In [36]:
id((A**1)), id((A**2).Net[0]), id((A**2).Net[1])  # 深拷贝

(23275240731600, 23275250182096, 23275267990736)

In [37]:
B = (A**A)**3  # 深拷贝
id(B.Net[0].Net[0]), id(B.Net[1].Net[0]), id(B.Net[2].Net[0])

(23275247281840, 23275247279632, 23275247225920)

In [38]:
id((A + A)), id((A + A).Net[0]), id((A + A).Net[1])  # 浅拷贝

(23275267984112, 23275250179840, 23275250179840)

In [39]:
id((A & A)), id((A & A).Net[0]), id((A & A).Net[1])  # 深拷贝

(23275240735888, 23275247280976, 23275247224192)

自适应调整输入形状

- 设置输入维度为 0，用 set_i()或在首次 forward 时自适应调整顶层输入形状
- 输入维度不为 0，用 set_i()可强制更新顶层输入形状

In [42]:
from macronet import *

F = Cell(1, 10, ['fc', 'act', 'fc', 'act', 'fc', 'act']) * layer(0, 1)
F(torch.tensor([[0.0]])), F

(tensor([[0.3071]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:fc): Linear(in_features=1, out_features=10, bias=True)
         (1:act): PReLU(num_parameters=1)
         (2:fc): Linear(in_features=10, out_features=10, bias=True)
         (3:act): PReLU(num_parameters=1)
         (4:fc): Linear(in_features=10, out_features=10, bias=True)
         (5:act): PReLU(num_parameters=1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=10, out_features=1, bias=True)
     )
   )
 ))

In [43]:
F.set_i(2)(torch.tensor([[0.0, 1.0]])), F # 强制更新

(tensor([[0.2324]], grad_fn=<AddmmBackward0>),
 MoNet(
   (Net): Sequential(
     (0): Cell(
       (Net): Sequential(
         (0:fc): Linear(in_features=2, out_features=10, bias=True)
         (1:act): PReLU(num_parameters=1)
         (2:fc): Linear(in_features=10, out_features=10, bias=True)
         (3:act): PReLU(num_parameters=1)
         (4:fc): Linear(in_features=10, out_features=10, bias=True)
         (5:act): PReLU(num_parameters=1)
       )
     )
     (1): MoNet(
       (Net): Linear(in_features=10, out_features=1, bias=True)
     )
   )
 ))